In [2]:
! pip install PyMySQL
! python3 -m pip install PyMySQL[rsa]
! pip3 install crytography

!pip install chart_studio
!pip install plotly==4.4.1
!pip install pytrends
!apt-get -qq install -y libarchive-dev && pip install -q -U libarchive

!apt-get -qq install python-cartopy python3-cartopy
!apt-get install -qq libgdal-dev libproj-dev
!pip install Cartopy==0.17.0

     |████████████████████████████████| 43 kB 1.1 MB/s 
     |████████████████████████████████| 3.0 MB 5.1 MB/s 
ERROR: Could not find a version that satisfies the requirement crytography (from versions: 0.0.1, 1.0.0)
ERROR: No matching distribution found for crytography
     |████████████████████████████████| 64 kB 2.1 MB/s 
Selecting previously unselected package libarchive-dev:amd64.
(Reading database ... 148489 files and directories currently installed.)
Preparing to unpack .../libarchive-dev_3.2.2-3.1ubuntu0.7_amd64.deb ...
Unpacking libarchive-dev:amd64 (3.2.2-3.1ubuntu0.7) ...
Setting up libarchive-dev:amd64 (3.2.2-3.1ubuntu0.7) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
     |████████████████████████████████| 154 kB 7.4 MB/s 
Selecting previously unselected package python-pkg-resources.
(Reading database ... 148545 files and directories currently installed.)
Preparing to unpack .../00-python-pkg-resources_39.0.1-2_all.deb ...
Unpacking python-pkg-resources (39.0.

In [3]:
! python3 -m pip install PyMySQL[rsa]

In [4]:
#! pip3 install crytography

In [5]:
import pymysql.cursors

In [6]:
connection = pymysql.connect(host = '15.165.15.157',
                             user = 'USER_SQLSTK',
                             password = '1qaz2wsx',
                             database = 'DB_SQLSTK',
                             charset = 'utf8',
                             cursorclass = pymysql.cursors.DictCursor
                             )

OperationalError: ignored

SQL 연결

In [ ]:
connection

In [ ]:
cursor = connection.cursor()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly
import plotly.express as px

삼성바이오로직스 주가 정보 SQL에서 가져오기

In [ ]:
sql = """
      SELECT T1.STk_CD, T1.DT, T1.C_PRC, T1.O_PRC, T1.H_PRC, T1.L_PRC, T1.VOL
      FROM HISTORY_DT T1
      WHERE T1.STK_CD = '207940'
      AND T1.DT >= '2019-01-01'
      AND T1.DT <= '2020-12-31'
"""

In [ ]:
cursor.execute(sql)

In [ ]:
result = cursor.fetchall()

In [ ]:
result

In [ ]:
df = pd.DataFrame(result)

In [ ]:
df

In [ ]:
df.index = df['DT']
df

In [ ]:
df["DT"] = pd.to_datetime (df["DT"])

In [ ]:
df.loc[:, "C_PRC" : ] = df.loc[: , "C_PRC" : ].astype(int)

머신러닝으로 주가 예측 그래프 그려보기(fbprophet 라이브러리 이용)

In [ ]:
! pip install chart_studio
! pip install plotly==4.4.1
! pip install pytrends
!apt-get -qq install -y libarchive-dev && pip install -q -U libarchive
import libarchive

import pydot
!apt-get -qq install python-cartopy python3-cartopy
!apt-get install -qq libgdal-dev libproj-dev
!pip install Cartopy==0.17.0

In [ ]:
import matplotlib.dates as mdates
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, add_changepoints_to_plot

import warnings
warnings.filterwarnings("ignore")

import cartopy
import chart_studio.plotly as py
import plotly.graph_objs as go
import plotly.express as px
import math
import seaborn as sns

from plotly.offline import iplot
from plotly.subplots import make_subplots
from plotly import graph_objects as go
from pytrends.request import TrendReq

In [ ]:
sql1 = """
      SELECT T2.DT Date
          , T2.O_PRC Open , T2.L_PRC Low, T2.H_PRC High, T2.C_PRC Close, T2.VOL Volume
          , T2.CHG_RT 등락율
      FROM stock T1
      INNER JOIN HISTORY_DT T2
      ON (T2.STK_CD = T1.STK_CD)=
      WHERE T1.STK_CD = '207940'
      AND T2.DT >= '2019-01-01'
      AND T2.DT <= '2020-05-15'
"""  

In [ ]:
cursor.execute(sql1)

In [ ]:
result_2 = cursor.fetchall()

In [ ]:
df_2 = pd.DataFrame(result_2)

In [ ]:
df_2["Date"] = pd.to_datetime(df_2["Date"])
df_2.loc[: , "Open":] = df_2.loc[: , "Open":].astype(int)

In [ ]:
df_prophet = df_2.rename(columns={
    'Date':'ds',
    'Close':'y'
})
df_prophet

In [ ]:
m = Prophet(
     yearly_seasonality=False,
     weekly_seasonality=False,
     daily_seasonality=True,
     seasonality_mode='additive'
 )

 m.fit(df_prophet)

 future = m.make_future_dataframe(periods=90)

 future.tail(90)

In [ ]:
forecast = m.predict(future)

forecast[['ds','yhat','yhat_lower','yhat_upper']].tail(90)

In [ ]:
m = Prophet(
    changepoint_prior_scale=0.2,  # 값이 커질수록 모델을 유연하게 만들어줌
    yearly_seasonality=False,
    weekly_seasonality=False,
    daily_seasonality=True,
    seasonality_mode='additive'
 )

m.fit(df_prophet)

future = m.make_future_dataframe(periods=90)

fig_1 = plot_plotly(m, forecast)

iplot(fig_1)

RSI 분석

In [ ]:
df["diff"] = df["C_PRC"].diff()

In [ ]:
def UD(diff):
    ups = 0
    downs =0
    if diff > 0:
      ups = diff
    elif diff < 0:
      downs = abs(diff)
    return pd.Series([ups, downs])

In [ ]:
df[["U", "D"]] = df["diff"].apply(UD)

In [ ]:
df

In [ ]:
df["AU"] = df["U"].rolling(14).mean()

In [ ]:
df["AD"] = df["D"].rolling(14).mean()

In [ ]:
df.tail()

In [ ]:
df["RS"] = df["AU"] / df["AD"]

In [ ]:
df["RSI"] = df["RS"] / (1 + df["RS"])

In [ ]:
df.tail()

In [ ]:
df["RSI_signal"] = df["RSI"].rolling(6).mean()

In [ ]:
df.tail()

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df

In [ ]:
fig = px.line(df, x='DT', y='C_PRC', title="삼성바이오로직스 일별 종가")
fig.show()

In [ ]:
df["C_PRC"].plot(figsize=(15,5))

In [ ]:
df[["RSI", "RSI_signal"]].plot(figsize=(15,5))
plt.axhline(0.7, c='r', ls="--")
plt.axhline(0.5, c='k', ls="--")
plt.axhline(0.3, c='b', ls="--")

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
1candle = go.Candlestick(x=df.index)
candle = go.Candlestick(
	x=df.index,
	open=df['O_PRC'],
	high=df['H_PRC'],
	low=df['L_PRC'],
	close=df['C_PRC'],
  increasing_line_color='red',
	decreasing_line_color='blue')

fig = go.Figure(data=candle)
fig.show()

In [ ]:
df

MACD 분석

In [ ]:
short = 12
df["ema_short"] = df["C_PRC"].ewm(short).mean()

In [ ]:
long = 26
df["ema_long"] = df["C_PRC"].ewm(long).mean()

In [ ]:
df = df.set_index("DT")

In [ ]:
df

In [ ]:
df[["C_PRC", "ema_short", "ema_long"]].plot(figsize=(15, 5))

In [ ]:
df["MACD"] = df["ema_short"] - df["ema_long"]

In [ ]:
df

In [ ]:
df["MACD"].plot(figsize=(15, 5))
plt.axhline(0)

In [ ]:
df["signal"] = df["MACD"].ewm(9).mean()

In [ ]:
df[["MACD", "signal"]].plot(figsize=(15, 5))
plt.axhline(0)

In [ ]:
df["macdhist"] = df["MACD"] - df["signal"]

In [ ]:
plt.figure(figsize=(15,5))
plt.bar(df.index, df.macdhist)
plt.plot(df.MACD)
plt.axhline(0)

In [ ]:
df["C_PRC"].plot(figsize=(15,5))

볼린저밴드 분석

In [ ]:
df["MA20"] = df["C_PRC"].rolling(20).mean()

In [ ]:
df["stddev"] = df["C_PRC"].rolling(20).std()

In [ ]:
df["UpperB"] = df["MA20"] + (df["stddev"] * 2)
df["LowerB"] = df["MA20"] - (df["stddev"] * 2)

In [ ]:
df

In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(df.index, df.UpperB, "r--")
plt.plot(df.index, df.C_PRC, "k")
plt.plot(df.index, df.LowerB, "b--")
plt.fill_between(df.index, df.LowerB, df.UpperB, alpha=0.2)

In [ ]:
df["PercentB"] = (df["C_PRC"] - df["LowerB"]) / (df["UpperB"] - df["LowerB"])

In [ ]:
plt.figure(figsize=(15, 5))
plt.bar(df.index, df["PercentB"])
plt.axhline(1, c="r", lw=1)
plt.axhline(0.5, c="b", lw=1)
plt.axhline(0, c="k", lw=1)

MFI 분석

In [ ]:
df = pd.DataFrame(result_2)
df.index = df['Date']
df = df.drop(['Date'], axis=1)
stock_data = df
stock_data

In [ ]:
stock_data.index = pd.to_datetime(stock_data.index)
stock_data.loc[: , "Open":] = stock_data.loc[: , "Open":].astype(int)

In [ ]:
! pip install -U finance-datareader

import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import FinanceDataReader as fdr
plt.style.use('fivethirtyeight')
warnings.filterwarnings('ignore')

stock_data = fdr.DataReader('207940', '2019', '2021')
stock_data

In [ ]:
plt.figure(figsize=(13, 5))
plt.plot(stock_data['Close'], label = "Close Price", lw = 1)
plt.title('Close Price')
plt.xlabel("Date")
plt.ylabel("Close Price")
plt.legend(loc='upper left')
plt.show()

In [ ]:
typical_price = (stock_data['Close']+stock_data['High']+stock_data['Low'])/3
money_flow = typical_price * stock_data['Volume'] ## (Money Flow = 평균가격 * 거래량)

positive_flow = []
negative_flow = []

for i in range(1, len(typical_price)):
  if typical_price[i] > typical_price[i-1]:
    positive_flow.append(money_flow[i-1])
    negative_flow.append(0)
  elif typical_price[i] < typical_price[i-1]:
    negative_flow.append(money_flow[i-1])
    positive_flow.append(0)
  else:
    positive_flow.append(0)
    negative_flow.append(0)

In [ ]:
positive_mf = []
negative_mf = []
period = 14

for i in range(period-1, len(positive_flow)):
  positive_mf.append(sum(positive_flow[i+1-period : i+1]))
for i in range(period-1, len(negative_flow)):
  negative_mf.append(sum(negative_flow[i+1-period : i+1]))  

In [ ]:
mfi = 100 * (np.array(positive_mf) / (np.array(positive_mf) + np.array(negative_mf)))

date = pd.DataFrame(stock_data.index)
date = date.drop(date.index[:14])
date

In [ ]:
stock_data2 = pd.DataFrame()
stock_data2['Date'] = date['Date']
stock_data2['MFI'] = mfi
stock_data2["Date"] = pd.to_datetime(stock_data2["Date"])
stock_data2.index = stock_data2['Date']

In [ ]:
plt.figure(figsize = (13, 5))
plt.plot(stock_data2['MFI'], label = 'MFI', lw = 2)
plt.axhline(10, linestyle = '--', color = 'orange')
plt.axhline(20, linestyle = '--', color = 'blue')
plt.axhline(80, linestyle = '--', color = 'blue')
plt.axhline(90, linestyle = '--', color = 'orange')
plt.title('MFI')
plt.ylabel('MFI Values')
plt.show()

In [ ]:
new_stock_data = pd.DataFrame()
new_stock_data = stock_data[period:]
new_stock_data['MFI'] = mfi

In [ ]:

def get_signal(data, high, low):
  buy_signal = []
  sell_signal = []

  for i in range(len(data['MFI'])):
    if data['MFI'][i] > high:
      buy_signal.append(np.nan)
      sell_signal.append(data['Close'][i])

    elif data['MFI'][i] < low:
      buy_signal.append(data['Close'][i])
      sell_signal.append(np.nan)

    else:
      buy_signal.append(np.nan)
      sell_signal.append(np.nan)  

  return (buy_signal, sell_signal)

In [ ]:
new_stock_data['Buy'] = get_signal(new_stock_data, 80, 20)[0]
new_stock_data['Sell'] = get_signal(new_stock_data, 80, 20)[1]

In [ ]:
plt.figure(figsize=(13, 5))
plt.plot(new_stock_data['Close'], label='Close Price', alpha = 0.5)
plt.scatter(new_stock_data.index, new_stock_data['Buy'], color = 'green', label='Buy Siganl', marker ='^', alpha = 1 )
plt.scatter(new_stock_data.index, new_stock_data['Sell'], color = 'red', label='Sell Siganl', marker ='v', alpha = 1 )
plt.title('Close Price')
plt.xlabel('Date')
plt.legend(loc = 'upper left')
plt.show()

OVB 분석

In [ ]:
ova_data = fdr.DataReader('207940', '2019', '2021')
ova_data

In [ ]:
plt.figure(figsize=(13, 5))
plt.plot(ova_data['Close'], label = "Close Price", lw = 1)
plt.title('Close Price')
plt.xlabel("Date", fontsize= 18)
plt.ylabel("Price(WON)")
plt.show()

In [ ]:
OBV = []
OBV.append(0)

for i in range(1, len(ova_data.Close)):
  if ova_data.Close[i] > ova_data.Close[i-1]:
    OBV.append(OBV[-1] + ova_data.Volume[i])
  elif ova_data.Close[i] < ova_data.Close[i-1]:
    OBV.append(OBV[-1] - ova_data.Volume[i])
  else:
    OBV.append(OBV[-1])

In [ ]:
ova_data['OBV'] = OBV
ova_data['OBV_EMA'] = ova_data['OBV'].ewm(span=20).mean()

In [ ]:
plt.figure(figsize=(13, 5))
plt.plot(ova_data['OBV'], label = "OBV", lw = 1, color='orange')
plt.plot(ova_data['OBV_EMA'], label = "OBV_EMA", lw = 1, color='purple')
plt.title('OBV, OBV_EMA')
plt.xlabel("Date", fontsize= 18)
plt.ylabel("Price (WON)")
plt.legend(loc='upper left')
plt.show()

In [ ]:
def buy_sell(signal, col1, col2):
  sigPriceBuy = []
  sigPriceSell = []
  flag = -1
  for i in range(0, len(signal)):
    if signal[col1][i] > signal[col2][i] and flag != 1:
      sigPriceBuy.append(signal['Close'][i])
      sigPriceSell.append(np.nan)
      flag = 1
    elif signal[col1][i] < signal[col2][i] and flag != 0:
      sigPriceSell.append(signal['Close'][i])
      sigPriceBuy.append(np.nan)
      flag = 0
    else:
      sigPriceSell.append(np.nan)
      sigPriceBuy.append(np.nan)

  return(sigPriceBuy, sigPriceSell)

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
x = buy_sell(ova_data, 'OBV', 'OBV_EMA')
ova_data['Buy_Signal_Price'] = x[0]
ova_data['Sell_Signal_Price'] = x[1]
ova_data

In [ ]:
plt.figure(figsize=(13, 5))
plt.plot(ova_data['Close'], label='Close Price', alpha = 0.5)
plt.scatter(ova_data.index, ova_data['Buy_Signal_Price'], color = 'green', label='Buy Siganl', marker ='^', alpha = 1 )
plt.scatter(ova_data.index, ova_data['Sell_Signal_Price'], color = 'red', label='Sell Siganl', marker ='v', alpha = 1 )
plt.title('Buy and Sell Signals')
plt.xlabel('Date')
plt.legend(loc = 'upper left')
plt.show()